[Reference](https://towardsdatascience.com/apriori-algorithm-implementation-using-optimized-approach-with-pandas-a72aacd990fe)

In [2]:
from itertools import combinations
from operator import itemgetter
import pandas as pd
from time import time

In [4]:
table = pd.read_csv('https://raw.githubusercontent.com/karankharecha/Big_Data_Algorithms/master/datasets/sampledata_numbers.csv')

In [5]:
table.head()

,items
0,"2,5,6,9,17,22,24,31,37,39,40,42,47"
1,"2,4,6,16,17,19,22,39,43,49,50"
2,"2,6,10,11,15,17,18,23,26,28,33,47"
3,"4,17,19,22,40,43,46"
4,"6,8,10,22,26,35,37,47"


In [3]:
def perform_apriori(data, support_count):

    single_items = (data['items'].str.split(",", expand=True))\
        .apply(pd.value_counts).sum(axis=1).where(lambda value: value > support_count).dropna()

    apriori_data = pd.DataFrame(
        {'items': single_items.index.astype(int), 'support_count': single_items.values, 'set_size': 1})

    data['set_size'] = data['items'].str.count(",") + 1

    data['items'] = data['items'].apply(lambda row: set(map(int, row.split(","))))

    single_items_set = set(single_items.index.astype(int))

    for length in range(2, len(single_items_set) + 1):
        data = data[data['set_size'] >= length]
        d = data['items'] \
            .apply(lambda st: pd.Series(s if set(s).issubset(st) else None for s in combinations(single_items_set, length))) \
            .apply(lambda col: [col.dropna().unique()[0], col.count()] if col.count() >= support_count else None).dropna()
        if d.empty:
            break
        apriori_data = apriori_data.append(pd.DataFrame(
            {'items': list(map(itemgetter(0), d.values)), 'support_count': list(map(itemgetter(1), d.values)),
             'set_size': length}), ignore_index=True)

    return apriori_data


if __name__ == '__main__':
    table = pd.read_csv('https://raw.githubusercontent.com/karankharecha/Big_Data_Algorithms/master/datasets/sampledata_numbers.csv')
    start = time()
    print(perform_apriori(data=table, support_count=500))
    print(time() - start)

       items  support_count  set_size
0         10          733.0         1
1         11          654.0         1
2         16          775.0         1
3         17         1378.0         1
4         19         1144.0         1
5          2          791.0         1
6         23          751.0         1
7         26          693.0         1
8         28         1139.0         1
9         32          828.0         1
10        33          884.0         1
11        37         1194.0         1
12        39          645.0         1
13         4          644.0         1
14        40          669.0         1
15        43         1339.0         1
16        46          720.0         1
17        47          793.0         1
18         6         1349.0         1
19   (37, 6)          636.0         2
20  (37, 43)          577.0         2
21  (37, 17)          660.0         2
22  (37, 19)          519.0         2
23  (37, 28)          527.0         2
24   (6, 43)          691.0         2
25   (6, 17)